In [1]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import datetime
import time
import csv
import psycopg2
from sqlalchemy import create_engine
import schedule

In [2]:
def get_token():
    load_dotenv('./.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    client_id = os.environ.get("X-ClientId")
    pass_key = os.environ.get("passKey")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [3]:
def update_estaciones(): #Esta función hace UPDATE de los datos que no cambian de las estaciones en la base de datos
    load_dotenv('./.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    stations = requests.get(url, headers = headers).json()
    date_and_time = datetime.datetime.now()
    date_and_time_formated = date_and_time.strftime("%Y-%m-%d %H:%M:%S")
    date_and_time_formated2 = date_and_time.strftime("%Y%m%d%H%M%S")
    estaciones = pd.DataFrame(stations["data"])
    estaciones["last_updated"] = date_and_time_formated
    estaciones[["longitude", "latitude"]] = estaciones["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
    estaciones = estaciones.drop(["activate", "virtualDelete", "tipo_estacionPBSC", "geofence", "activate", "geometry", "integrator", "virtual_bikes", "virtual_bikes_num", "geofenced_capacity", "bikesGo"], axis=1)
    estaciones['coordinates'] = list(zip(estaciones['longitude'], estaciones['latitude']))
    update_estaciones = estaciones[["address", "code_district", "code_suburb", "id", "number", "total_bases", "last_updated", "longitude", "latitude", "coordinates"]]
    update_estaciones.to_csv(f'../data_csv/estaciones_{date_and_time_formated2}.csv')
    engine = create_engine("postgresql+psycopg2://postgres:"+ os.environ.get('postgre') +"@localhost/bicimad_worker")
    update_estaciones.to_sql('estaciones', con=engine, if_exists='replace', index=False)
    return update_estaciones

In [4]:
def get_disponibilidad(): #Esta función hace append en la base de datos de aquellas columnas que sí cambian a lo largo del día
    load_dotenv('./.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    stations = requests.get(url, headers = headers).json()
    date_and_time = datetime.datetime.now()
    date_and_time_formated = date_and_time.strftime("%Y-%m-%d %H:%M:%S")
    date_and_time_formated2 = date_and_time.strftime("%Y%m%d%H%M%S")
    estaciones = pd.DataFrame(stations["data"])
    estaciones["last_updated"] = date_and_time_formated
    estaciones[["longitude", "latitude"]] = estaciones["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
    estaciones = estaciones.drop(["virtualDelete", "tipo_estacionPBSC", "geofence", "geometry", "integrator", "virtual_bikes", "virtual_bikes_num", "geofenced_capacity", "bikesGo"], axis=1)
    estaciones['coordinates'] = list(zip(estaciones['longitude'], estaciones['latitude']))
    disponibilidad = estaciones[["activate","dock_bikes", "free_bases", "id", "light", "no_available", "reservations_count", "last_updated"]]
    disponibilidad.to_csv(f'../data_csv/disponibilidad_{date_and_time_formated2}.csv')
    engine = create_engine("postgresql+psycopg2://postgres:"+ os.environ.get('postgre') +"@localhost/bicimad_worker")
    disponibilidad.to_sql('disponibilidad', con=engine, if_exists='append', index=False)
    return disponibilidad

In [5]:
#Función para recopilar data en tiempo real sobre las estaciones de bicimad y poder almacenarla en una database
"""
def get_stations():
    load_dotenv('./.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    stations = requests.get(url, headers = headers).json()
    date_and_time = datetime.datetime.now()
    date_and_time_formated = date_and_time.strftime("%Y-%m-%d %H:%M:%S")
    date_and_time_formated2 = date_and_time.strftime("%Y%m%d%H%M%S")
    stations_real_time = pd.DataFrame(stations["data"])
    stations_real_time["last_updated"] = date_and_time_formated
    stations_real_time[["longitude", "latitude"]] = stations_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
    stations_real_time = stations_real_time.drop(["activate", "virtualDelete", "tipo_estacionPBSC", "geofence", "activate", "geometry", "integrator", "virtual_bikes", "virtual_bikes_num", "geofenced_capacity", "bikesGo"], axis=1)
    stations_real_time['coordinates'] = list(zip(stations_real_time['longitude'], stations_real_time['latitude']))
    stations_real_time.to_csv(f'../data_csv/stations_{date_and_time_formated2}.csv')
    engine = create_engine("postgresql+psycopg2://postgres:"+ os.environ.get('postgre') +"@localhost/bicimad_worker")
    stations_real_time.to_sql('stations', con=engine, if_exists='append', index=False)
    return stations_real_time
"""

'\ndef get_stations():\n    load_dotenv(\'./.env\')\n    token = os.environ.get("access_token")\n    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"\n    headers = {"accessToken" : token}\n    stations = requests.get(url, headers = headers).json()\n    date_and_time = datetime.datetime.now()\n    date_and_time_formated = date_and_time.strftime("%Y-%m-%d %H:%M:%S")\n    date_and_time_formated2 = date_and_time.strftime("%Y%m%d%H%M%S")\n    stations_real_time = pd.DataFrame(stations["data"])\n    stations_real_time["last_updated"] = date_and_time_formated\n    stations_real_time[["longitude", "latitude"]] = stations_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))\n    stations_real_time = stations_real_time.drop(["activate", "virtualDelete", "tipo_estacionPBSC", "geofence", "activate", "geometry", "integrator", "virtual_bikes", "virtual_bikes_num", "geofenced_capacity", "bikesGo"], axis=1)\n    stations_real_time[\'coordinates\'] = list(zip(stations_

In [6]:
get_disponibilidad()

,activate,dock_bikes,free_bases,id,light,no_available,reservations_count,last_updated
0,1,24,3,1406,1,0,0,2024-02-26 12:15:11
1,1,12,7,1407,2,0,0,2024-02-26 12:15:11
2,1,14,13,1408,2,0,0,2024-02-26 12:15:11
3,1,16,11,1409,2,0,0,2024-02-26 12:15:11
4,1,4,15,1410,0,0,0,2024-02-26 12:15:11
...,...,...,...,...,...,...,...,...
608,1,0,0,2368,3,1,0,2024-02-26 12:15:11
609,1,10,14,2061,2,0,0,2024-02-26 12:15:11
610,1,20,7,1530,1,0,0,2024-02-26 12:15:11
611,1,13,10,2305,2,0,0,2024-02-26 12:15:11


In [7]:
#Esto hace que se ejecute la función cada hora y se carguen los datos en mi base de datos
schedule.every().hour.do(get_disponibilidad)

while True:
    schedule.run_pending()
    time.sleep(1)

schedule.every().month.do(update_estaciones)

while True:
    schedule.run_pending()
    time.sleep(1)

ConnectionError: HTTPSConnectionPool(host='openapi.emtmadrid.es', port=443): Max retries exceeded with url: /v3/transport/bicimad/stations/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fcdd66be940>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))